In [1]:
import json
from functools import partial
from multiprocessing import Pool
import pandas as pd
import pymysql
import requests
from tqdm.auto import tqdm

In [2]:
def translate(query):
    data = {
        'q': query,
        'target': 'fr',
        'format': 'text',
        'source': 'en'
    }
    try:
        response = requests.post('https://translation.googleapis.com/language/translate/v2?key=', data=json.dumps(data))
        if response.status_code == 200:
            return [response.json()['data']['translations'][0]['translatedText'], query]
        return ['ERROR', query]
    except Exception as ex:
        print(str(ex))
        return ['EXCEPT', query]

In [3]:
def translate_list():
    conn = pymysql.connect(host='192.168.1.44', port=3306, user='euclidlab', passwd='1q2w3e4r!', db='vqa_france',
                           charset='utf8', autocommit=True)
    with conn.cursor() as curs:
        sql = """
select distinct a.en
from (select answer as en
      from data_answer
      where data_answer.answer_kor is null) a
where not exists(
    select 1 from translate_word b where a.en = b.en
  )
        """
        curs.execute(sql)
        return curs.fetchall()

In [4]:
def translate_insert(ko, en):
    conn = pymysql.connect(host='192.168.1.44', port=3306, user='euclidlab', passwd='1q2w3e4r!', db='vqa_france',
                           charset='utf8', autocommit=True)
    with conn.cursor() as curs:
        sql = "insert into translate_word(ko, en) values (%s, %s)"
        curs.execute(sql, (ko, en))

In [5]:
dup_questions = pd.DataFrame(translate_list(), columns=['en'])

In [6]:
dup_questions

,en
0,semi circle
1,double curve
2,twisting
3,i have no idea
4,;photo
...,...
111657,13 +
111658,at bicycle stand
111659,question-mark
111660,bottom rim


In [ ]:
with Pool(8) as pool:
    result = list(tqdm(pool.imap(
        func=partial(translate),
        iterable=dup_questions['en']
    ), total=len(dup_questions['en'])))
    pool.close()
    pool.join()

In [ ]:
#translate('Is there a shadow?', 'en', 'fr', False)

In [ ]:
for idx, row in enumerate(result):
    ko = row[0]
    en = row[1]

    translate_insert(ko, en)